In [92]:
runs -y rm %

In [93]:
rm -f runs.db .runsrc

# Starting out

We begin by creating a new `.runsrc` file and creating our database `runs.db`. Running any `runs` command will do this. Note that without `--assume-yes`, this would prompt the user for confirmation. You can also use the short flag `-y`.

In [94]:
runs --assume-yes ls

Config not found. Using default config:
{'args': '',
 'db_path': '/Users/ethan/lab-notebook/demo/runs.db',
 'dir_names': '',
 'root': '/Users/ethan/lab-notebook/demo/.runs'}



In [95]:
ls runs.db

runs.db


In [96]:
cat .runsrc

[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : 
args : 



The `.runsrc` establishes various defaults for the `runs` command although these can be overriden with command line arguments. We will explore some of the capabilities of this file later in the tutorial.

# Creating runs

In [97]:
runs -y new --path=demo --command=python --description="Demonstrate lab-notebook capabilities."

Path:
demo
Description:
Demonstrate lab-notebook capabilities.
Command sent to session:
python
List active:
tmux list-session
Attach:
tmux attach -t demo



In [98]:
runs ls demo

demo


You can query the database about runs:

In [99]:
runs lookup command demo

demo: python


In [100]:
runs lookup datetime demo

demo: 2018-12-25T11:33:35.123694


You can create multiple runs with a single run command:

In [101]:
runs -y new --path=demo --command="python -c 'print(1)'" \
            --path=demo2 --command="python -c 'print(2)'" \
            --description="Demonstrate lab-notebook capabilities."

Path:
demo2
Description:
Demonstrate lab-notebook capabilities.
Command sent to session:
python -c 'print(2)'
List active:
tmux list-session
Attach:
tmux attach -t demo2

Path:
demo
Description:
Demonstrate lab-notebook capabilities.
Command sent to session:
python -c 'print(1)'
List active:
tmux list-session
Attach:
tmux attach -t demo



In [102]:
runs ls dem%

demo2
demo


In [103]:
tmux ls | grep demo

demo: 1 windows (created Tue Dec 25 11:33:36 2018) [80x24]
demo2: 1 windows (created Tue Dec 25 11:33:36 2018) [80x24]


Note that `demo` has been overwritten. Previously the command for `demo` was `python`. Now it is:

In [104]:
runs lookup command demo

demo: python -c 'print(1)'


In [105]:
runs ls --active

demo
demo2


You can also filter by time.

In [106]:
runs ls --since $(date "+%Y-%m-%d")

demo2
demo


In [107]:
runs ls --from-last 1day

demo2
demo


For info on accepted formats, run `runs ls -h` (ommitted for brevity).

All these selection mechanisms are available to other `runs` subcommands (`rm`, `mv`, `ls`, `lookup`, `change-description`, `reproduce`, `correlate`, `kill`).

# Changing runs

In [108]:
runs -y mv demo demo1

In [109]:
runs ls --active

demo1
demo2


In [110]:
tmux ls

0: 1 windows (created Tue Dec 25 10:56:38 2018) [174x45]
demo1: 1 windows (created Tue Dec 25 11:33:36 2018) [80x24]
demo2: 1 windows (created Tue Dec 25 11:33:36 2018) [80x24]


In [111]:
runs change-description --active 'A new description'

In [112]:
runs lookup description --active

demo1: A new description
demo2: A new description


We can also kill the TMUX session for runs without deleting the database record:

In [114]:
runs -y kill --active

In [115]:
tmux ls | grep demo  # no runs match

: 1

In [116]:
runs ls --active  # no runs found

In [117]:
runs ls demo%  # but runs were not deleted from database

demo2
demo1


# Using the `.runsrc`

In this section we will focus on two fields in the `.runsrc`:
* `dir_names` specifies directories that will be placed inside `root` and will be synchronized with run paths (created, moved, and deleted with them).
* `args` specifies flags that should be passed to every run

In [119]:
cat "[main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : write-dir
args : --write-path=${main:root}/write-dir/<path>" > .runsrc

cat: [main]
root : /Users/ethan/lab-notebook/demo/.runs
db_path : /Users/ethan/lab-notebook/demo/runs.db
dir_names : write-dir
args : --write-path=/write-dir/<path>: No such file or directory


: 1

There are a couple things to note here:
* `<path>` will be replaced with the path of each run.
* 
